Problem 4.1

In [121]:
import pandas as pd
import numpy as np

data_path = 'titanic_data.csv'
df = pd.read_csv(data_path)

df['class1'] = (df['Pclass'] == 1).astype(int)
df['class2'] = (df['Pclass'] == 2).astype(int)
df['isFemale'] = df['Sex']
df['isMinor'] = (df['Age'] < 18).astype(int)
df['withSiSp'] = (df['Siblings/Spouses Aboard'] > 0).astype(int)
df['withPaCh'] = (df['Parents/Children Aboard'] > 0).astype(int)
median_fare = df['Fare'].median()
df['highFare'] = (df['Fare'] > median_fare).astype(int)

df.drop(['Age','Sex','Pclass', 'Siblings/Spouses Aboard', 'Parents/Children Aboard','Fare'], axis=1, inplace=True)
print(df.head())

   Survived  class1  class2  isFemale  isMinor  withSiSp  withPaCh  highFare
0         0       0       0         0        0         1         0         0
1         1       1       0         1        0         1         0         1
2         1       0       0         1        0         0         0         0
3         1       1       0         1        0         1         0         1
4         0       0       0         0        0         0         0         0


Problem 4.2

In [122]:
def mutual_information(xj, y):
    xj = np.array(xj)
    y = np.array(y)

    p_xj_1 = np.mean(xj == 1)
    p_xj_0 = 1 - p_xj_1
    p_y_1 = np.mean(y == 1)
    p_y_0 = 1 - p_y_1

    p_xj_0_y_1 = np.mean((xj == 0) & (y == 1))
    p_xj_0_y_0 = np.mean((xj == 0) & (y == 0))
    p_xj_1_y_1 = np.mean((xj == 1) & (y == 1))
    p_xj_1_y_0 = np.mean((xj == 1) & (y == 0))
   
    mI = 0
    if p_xj_1_y_1 > 0: mI += p_xj_1_y_1 * np.log2(p_xj_1_y_1 / (p_xj_1 * p_y_1))
    if p_xj_1_y_0 > 0: mI += p_xj_1_y_0 * np.log2(p_xj_1_y_0 / (p_xj_1 * p_y_0))
    if p_xj_0_y_1 > 0: mI += p_xj_0_y_1 * np.log2(p_xj_0_y_1 / (p_xj_0 * p_y_1))
    if p_xj_0_y_0 > 0: mI += p_xj_0_y_0 * np.log2(p_xj_0_y_0 / (p_xj_0 * p_y_0))

    return mI

Problem 4.3

In [153]:
class decision_tree_node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class decision_tree:
    def __init__(self, maxDepth=5, minSamplesSplit=10):
        self.root = None
        self.maxDepth = maxDepth
        self.minSamplesSplit = minSamplesSplit

    def fit(self, X, y):
        self.root = self._grow_tree(X, y, depth=0)

    def predict(self, X):
        return np.array([self._predict(inputs, self.root) for inputs in X])

    def _predict(self, inputs, node):
        if node.value is not None:
            return node.value
        if inputs[node.feature] == 0:
            return self._predict(inputs, node.left)
        return self._predict(inputs, node.right)
        
    def _grow_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        if n_samples == 0 or depth >= self.maxDepth or n_samples < self.minSamplesSplit or len(np.unique(y)) == 1:
            leaf_value = self._most_common_label(y)
            return decision_tree_node(value=leaf_value)

        best_feature, _ = self._best_split(X, y, n_features)
        
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return decision_tree_node(value=leaf_value)

        left_idxs, right_idxs = X[:, best_feature] == 0, X[:, best_feature] == 1
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)
        left = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        return decision_tree_node(feature=best_feature, left=left, right=right)

    def _best_split(self, X, y, n_features):
        best_mi = -1
        split_idx = None
        for feature_idx in range(n_features):
            mi = mutual_information(X[:, feature_idx], y)
            if mi > best_mi:
                best_mi = mi
                split_idx = feature_idx
        return split_idx, best_mi

    def _most_common_label(self, y):
        if len(y) == 0:
            return 0
        else:
            return np.bincount(y).argmax()

    

Problem 4.4

In [154]:
def print_tree(node, depth=0, feature_names=df.drop('Survived', axis=1).columns, excluded_feature=None):
    prefix = "\t" * depth
    if node.value is not None:
        print(f"{prefix}Predict: {node.value}")
    else:
        if node.feature == excluded_feature:
            print(f"{prefix}Feature '{feature_names[node.feature]}' (excluded)")
        else:
            print(f"{prefix}Feature '{feature_names[node.feature]}'?")
        print(f"{prefix}=> True:")
        print_tree(node.left, depth + 1, feature_names, excluded_feature)
        print(f"{prefix}=> False:")
        print_tree(node.right, depth + 1, feature_names, excluded_feature)

In [155]:
X = df.drop('Survived', axis=1).values
y = df['Survived'].values

tree = decision_tree()
tree.fit(X, y)

feature_names = list(df.drop('Survived', axis=1).columns)
print_tree(tree.root, feature_names=feature_names)

Feature 'isFemale'?
=> True:
	Feature 'class1'?
	=> True:
		Feature 'isMinor'?
		=> True:
			Feature 'withPaCh'?
			=> True:
				Feature 'class2'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
			=> False:
				Feature 'class2'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
		=> False:
			Feature 'class2'?
			=> True:
				Feature 'withPaCh'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
			=> False:
				Feature 'withPaCh'?
				=> True:
					Predict: 0
				=> False:
					Predict: 1
	=> False:
		Feature 'isMinor'?
		=> True:
			Feature 'highFare'?
			=> True:
				Predict: 0
			=> False:
				Feature 'withPaCh'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
		=> False:
			Predict: 1
=> False:
	Feature 'class1'?
	=> True:
		Feature 'class2'?
		=> True:
			Feature 'withPaCh'?
			=> True:
				Feature 'isMinor'?
				=> True:
					Predict: 1
				=> False:
					Predict: 1
			=> False:
				Feature 'withSiSp'?
				=> True:
					Predict: 1
				=> F

Problem 4.5

In [126]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=42)

accuracies = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    decision_tree = decision_tree(maxDepth=3, minSamplesSplit=20)
    decision_tree.fit(X_train, y_train)
    
    y_pred = decision_tree.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f"Average Accuracy of 10-Fold Cross-Validation : {average_accuracy:.4f}")


Average 10-Fold Cross-Validation Accuracy: 0.7779


Problem 4.6

In [127]:
x = [0, 1, 0, 0, 0, 1, 1] #class1  class2  isFemale  isMinor  withSiSp  withPaCh  highFare
prediction = decision_tree.predict(np.array([x]))[0]

if prediction == 1:
    print("I survive the Titanic.")
else:
    print("I dont survive Titanic.")


I dont survive Titanic.


Problem 4.7

In [128]:
from sklearn.utils import resample
from scipy.stats import mode
class random_forest:
    def __init__(self, n_trees=5, maxDepth=5, minSamplesSplit=10, sample_size=0.8):
        self.n_trees = n_trees
        self.maxDepth = maxDepth
        self.minSamplesSplit = minSamplesSplit
        self.sample_size = sample_size
        self.trees = []

    def fit(self, X, y):
        n_features = X.shape[1]
        for _ in range(self.n_trees):
            sample_X, sample_y = resample(X, y, n_samples=int(self.sample_size * len(y)))
            tree = decision_tree(maxDepth=self.maxDepth, minSamplesSplit=self.minSamplesSplit)
            tree.fit(sample_X, sample_y)
            self.trees.append(tree)

    def predict(self, X):
        predictions = [tree.predict(X) for tree in self.trees]
        predictions = np.array(predictions)
        final_prediction, _ = mode(predictions, axis=0)
        return np.squeeze(final_prediction)


In [129]:
def display_random_forest(random_forest, feature_names):
    for i, tree in enumerate(random_forest.trees):
        print(f"Tree {i + 1}:")
        print_tree(tree.root, depth=0, feature_names=feature_names)
        print("-" * 50)

In [130]:
feature_names = ['class1', 'class2', 'isFemale', 'isMinor', 'withSiSp', 'withPaCh', 'highFare']
A = df.drop('Survived', axis=1).values
b = df['Survived'].values
rf_model = random_forest()
rf_model.fit(A, b)
display_random_forest(rf_model, feature_names)

Tree 1:
Feature 'isFemale'?
=> True:
	Feature 'class1'?
	=> True:
		Feature 'isMinor'?
		=> True:
			Feature 'withSiSp'?
			=> True:
				Feature 'highFare'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
			=> False:
				Predict: 0
		=> False:
			Feature 'class2'?
			=> True:
				Feature 'withPaCh'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
			=> False:
				Feature 'withPaCh'?
				=> True:
					Predict: 0
				=> False:
					Predict: 1
	=> False:
		Feature 'isMinor'?
		=> True:
			Feature 'withPaCh'?
			=> True:
				Feature 'highFare'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
			=> False:
				Feature 'withSiSp'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
		=> False:
			Predict: 1
=> False:
	Feature 'class1'?
	=> True:
		Feature 'class2'?
		=> True:
			Feature 'withSiSp'?
			=> True:
				Feature 'withPaCh'?
				=> True:
					Predict: 1
				=> False:
					Predict: 0
			=> False:
				Feature 'withPaCh'?
				=> True:
					Predic

In [136]:
kf2 = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in kf2.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    forest = random_forest()
    forest.fit(X_train, y_train)
    
    y_pred = forest.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f"Average Accuracy of 10-Fold Cross-Validation : {average_accuracy:.4f}")

Average 10-Fold Cross-Validation Accuracy: 0.8083


In [137]:
k = [0, 1, 0, 0, 0, 1, 1] #class1  class2  isFemale  isMinor  withSiSp  withPaCh  highFare
prediction = forest.predict(np.array([k]))

if prediction == 1:
    print("I survive the Titanic.")
else:
    print("I dont survive Titanic.")

I dont survive Titanic.


Problem 4.8

In [138]:
class random_forest_excluding_features:
    def __init__(self, n_trees=6, maxDepth=5, minSamplesSplit=10, sample_size=0.8):
        self.n_trees = n_trees
        self.maxDepth = maxDepth
        self.minSamplesSplit = minSamplesSplit
        self.sample_size = sample_size
        self.trees = []
    
    def predict(self, X):
        tree_predictions = []
        for tree, included_features in self.trees:
            predictions = tree.predict(X[:, included_features])
            tree_predictions.append(predictions)
        tree_predictions = np.array(tree_predictions)
        final_prediction, _ = mode(tree_predictions, axis=0)
        return np.squeeze(final_prediction)
    
    def fit(self, X, y):
        n_features = X.shape[1]
        # Excludes the class1 & class2 together, then each of the other feature
        feature_combinations = [[0, 1]] + [[i] for i in range(2, n_features)]  
        for excluded_features in feature_combinations[:self.n_trees]:
            included_features = [i for i in range(n_features) if i not in excluded_features]
            sample_X, sample_y = resample(X[:, included_features], y, n_samples=int(self.sample_size * len(y)))
            tree = decision_tree(maxDepth=self.maxDepth, minSamplesSplit=self.minSamplesSplit)
            tree.fit(sample_X, sample_y)
            self.trees.append((tree, included_features))

    

In [146]:
#b
def cross_validate_random_forest(X, y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        rf = random_forest_excluding_features()
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        accuracy = np.mean(y_pred == y_test)
        accuracies.append(accuracy)
    average_accuracy = np.mean(accuracies)
    return average_accuracy

T = A
u = b

average_accuracy = cross_validate_random_forest(T, u)
print(f"Average Accuracy of 10-Fold Cross-Validation : {average_accuracy:.4f}")


Average accuracy from 10-fold cross-validation: 0.8117


In [151]:
#a
def display_random_forest_excluding_features(random_forest, feature_names):
    for i, (tree, included_features) in enumerate(random_forest.trees):
        print(f"Tree {i + 1} (Excluding features: {[feature_names[idx] for idx in range(len(feature_names)) if idx not in included_features]}):")
        print_tree(tree.root, depth=0, feature_names=[feature_names[idx] for idx in included_features])
        print("-" * 50)


In [152]:
rf = random_forest_excluding_features()
rf.fit(T, u)
display_random_forest_excluding_features(rf, feature_names)


Tree 1 (Excluding features: ['class1', 'class2']):
Feature 'isFemale'?
=> True:
	Feature 'isMinor'?
	=> True:
		Feature 'highFare'?
		=> True:
			Feature 'withSiSp'?
			=> True:
				Feature 'isFemale'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
			=> False:
				Feature 'withPaCh'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
		=> False:
			Feature 'withPaCh'?
			=> True:
				Feature 'withSiSp'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
			=> False:
				Feature 'withSiSp'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
	=> False:
		Feature 'withPaCh'?
		=> True:
			Feature 'withSiSp'?
			=> True:
				Predict: 0
			=> False:
				Predict: 0
		=> False:
			Feature 'withSiSp'?
			=> True:
				Predict: 1
			=> False:
				Feature 'highFare'?
				=> True:
					Predict: 0
				=> False:
					Predict: 0
=> False:
	Feature 'withSiSp'?
	=> True:
		Feature 'highFare'?
		=> True:
			Feature 'withPaCh'?
			=> True:
				Feature 'isMinor'?
				=

In [150]:
#c
s = [0, 1, 0, 0, 0, 1, 1]
prediction = rf.predict(np.array([s]))

if prediction == 1:
    print("I survive the Titanic.")
else:
    print("I dont survive Titanic.")

I dont survive Titanic.
